In [2]:
# Import libs
import numpy as np
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

In [ ]:
CLIENT_ID = 'QKXLFOGEM2RLE3AHMICC2ZHUXOZDWTFT1HLK0IP4O0MTWAUT'
CLIENT_SECRET = 'OCOYFA0RMR0M4VGKAJQZDZV50GV2MAGPZAWTJZT3MIKG2BXA'
VERSION = '20180604'
LIMIT = 200
search_query = "swimming pool"
radius = 2000

In [3]:
# Extract the original data frame 

df_data = pd.read_csv('Toronto Dataset.csv')
df_data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
# Cleaning the data
Cleaned_df = df_data[df_data.Borough != 'Not assigned']
mask = Cleaned_df.Neighborhood == 'Not assigned'
Cleaned_df.Neighborhood[mask] = Cleaned_df.Borough
Cleaned_df.shape

(103, 3)

In [10]:
#  Import and merge Coordinates 
df_data_latlng = pd.read_csv('Geospatial_Coordinates.csv')
neighborhoods = Cleaned_df.merge(df_data_latlng, on='Postal Code')

neighborhoods.to_csv('Toronto Dataset latlng.csv')
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [11]:
# Filter Toronto neighborhoods

toronto_Neighboorhoods = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")] 

In [12]:
# Get the latlng of Toronto

address = 'Toronto, Ontario, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [13]:
# Draw the map with neighboorhood without Clustering

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_Neighboorhoods['Latitude'], toronto_Neighboorhoods['Longitude'], toronto_Neighboorhoods['Borough'], toronto_Neighboorhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
# Identify Number of Unique Borough
print('There are {} uniques Borough.'.format(len(toronto_Neighboorhoods['Borough'].unique())))

There are 4 uniques Borough.


In [16]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)
# We will go with 4 clusters

kclusters = 4

# x = np.arange(kclusters)
# ys = [i + x + (i*x)**2 for i in range(kclusters)]
# colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))

# Create color_palette 
colors_array = sns.color_palette()
rainbow = [colors.rgb2hex(i) for i in colors_array]

colors_array
rainbow

markers_colors = []
Clusters = list(toronto_Neighboorhoods['Borough'].unique())
rainbow[Clusters.index('Downtown Toronto')]
for lat, lon, poi, cluster in zip(toronto_Neighboorhoods['Latitude'], toronto_Neighboorhoods['Longitude'], toronto_Neighboorhoods['Neighborhood'], toronto_Neighboorhoods['Borough']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[Clusters.index(cluster)],
        fill=True,
        fill_color=rainbow[Clusters.index(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
    
    
    
       
map_clusters